# Dataframe Generator

In [1]:
import pandas as pd
import sys
sys.path.append("../")
from Scripts.eda_Read_data import ReadData
from Scripts.cleaning import CleanDataFrame

In [2]:
# analyzer = Analysis()
cleaner = CleanDataFrame()
# plotter = Plotters(w=7, h=5)

In [4]:
loader = ReadData()
df = loader.read_csv('../Data/AdSmartABdata.csv')

In [5]:
df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077 entries, 0 to 8076
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   auction_id   8077 non-null   object
 1   experiment   8077 non-null   object
 2   date         8077 non-null   object
 3   hour         8077 non-null   int64 
 4   device_make  8077 non-null   object
 5   platform_os  8077 non-null   int64 
 6   browser      8077 non-null   object
 7   yes          8077 non-null   int64 
 8   no           8077 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 568.0+ KB


Next, I will use the cleaner pipeline to clean the data.

- **drop_duplicates**: looks for duplicated auction_ids and removes the row
- **drop_unresponsive**: drops rows where `yes` and `no` are 0
- **date_to_day**: Converts the date to the day of the week
- **convert_to_brands**: Converts the device_make (model) to a binary of known or generic brand
- **merge_response_columns**: Merges one-hot-encoded reponse column
- **drop_columns**: drops columns that are not needed or became obsolete

All the above are implemented in the CleanDataFrame class. I will user the `run_pipeline` method to pass our data through them.

In [7]:
df = cleaner.run_pipeline(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1243 entries, 0 to 1242
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   experiment   1243 non-null   object
 1   hour         1243 non-null   int64 
 2   platform_os  1243 non-null   int64 
 3   browser      1243 non-null   object
 4   day_of_week  1243 non-null   object
 5   brand        1243 non-null   object
 6   response     1243 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 68.1+ KB


In [ ]:
df.head()

,experiment,hour,platform_os,browser,day_of_week,brand,response
0,exposed,2,6,Chrome Mobile WebView,Sunday,generic,0
1,exposed,16,6,Chrome Mobile,Saturday,generic,1
2,exposed,8,6,Chrome Mobile,Monday,generic,0
3,control,4,6,Facebook,Wednesday,known brand,1
4,control,15,6,Chrome Mobile,Friday,generic,0


In [16]:
df.to_csv("../Data/AdSmartABdata.csv")

After this I will start creating the different version of the dataset.
This versions will be based on the `browser` and `platform_os`

## browsers

In [12]:
top_four_browsers = df.groupby('browser')['experiment'].count().nlargest(4)
top_four_browsers

browser
Chrome Mobile            695
Chrome Mobile WebView    227
Facebook                 156
Samsung Internet         145
Name: experiment, dtype: int64

I have selected the top 4 browsers

In [21]:
browser_dfs = {}
for browser in top_four_browsers.index:
    b_df = df.query(f"browser=='{browser}'").reset_index(drop=True)
    b_df.drop(columns=['browser'], inplace=True)
    file_path = f"../Data/browser_{browser.replace(' ', '_').lower()}.csv"
    print(file_path)
    b_df.to_csv(file_path, index=False)


../Data/browser_chrome_mobile.csv
../Data/browser_chrome_mobile_webview.csv
../Data/browser_facebook.csv
../Data/browser_samsung_internet.csv


At this point I am running the dvc add command and commiting the *.dvc files in Data


In [31]:
for os_name in df.platform_os.unique():
    os_df = df.query(f"platform_os=={os_name}")
    os_df = os_df.drop(columns=['platform_os'])
    file_path = f"../Data/platform_os_{os_name}.csv"
    os_df.to_csv(file_path, index=False)

In [32]:
os_df.head()

,experiment,hour,browser,day_of_week,brand,response
159,exposed,8,Mobile Safari,Friday,known brand,0
371,control,15,Mobile Safari,Friday,known brand,0
372,control,15,Mobile Safari UI/WKWebView,Friday,known brand,0
467,control,3,Mobile Safari,Saturday,known brand,1
558,exposed,3,Mobile Safari,Sunday,known brand,1
